In [101]:
import numpy as np
import random

In [25]:
def tokens_2(input_string):
    if len(input_string) % 2 != 0:
        raise ValueError("Input string length must be even.")
    tokens = [input_string[i:i+2] for i in range(0, len(input_string), 2)]  
    return tokens

In [26]:
class card:
    def __init__(self, rank, color, name):
        self.name = name
        self.rank = rank
        self.color = color
    def index(self):
        return (self.rank, self.color)


In [90]:
class map:
    def __init__(self):
        self.color_names = ['h', 's', 'd', 'c'] #columns
        self.rank_names = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2'] #rows
        self.card_map = {}
        for index_row in range(len(self.rank_names)):
            for index_col in range(len(self.color_names)):
                key = f"{self.rank_names[index_row]}{self.color_names[index_col]}"
                value = card(12-index_row, index_col, f"{self.rank_names[index_row]}{self.color_names[index_col]}")
                self.card_map[key] = value
    
    def card(self, name):
        return self.card_map[name]
    
    def card_index(self, name):
        return self.card_map[name].index()


In [92]:
card_map = map()
print(card_map.card_index('Ah'))

(12, 0)


In [142]:
class hero_cards:
    card_map = map()
    def __init__(self, cards = '', blocked_cards = ''):
        #print(hero_cards.card_map.card_map)
        self.card_array = np.zeros((13, 4))
        self.cards_names = tokens_2(cards)
        self.blocked_cards_names = tokens_2(blocked_cards)
        for name in self.cards_names:
            self.card_array[hero_cards.card_map.card_index(name)] = 1
        for name in self.blocked_cards_names:
            self.card_array[hero_cards.card_map.card_index(name)] = -1
    def deal_random(self):
        while True:
            random_rank = random.randint(0, 12)
            random_color = random.randint(0, 3)
            if self.card_array[random_rank, random_color] == 0:
                self.card_array[random_rank, random_color] = 1
                break
            
    def deal_card(self, name):
        if self.card_array[hero_cards.card_map.card_index(name)] == 1:
            self.card_array[hero_cards.card_map.card_index(name)] = 1

In [126]:
cards = hero_cards("2h3h4h5h6h")
array = cards.card_array
np.sum(array[0:0+5,:],axis = 0)

2h
3h
4h
5h
6h


array([5., 0., 0., 0.])

In [133]:
print(np.vstack((array[0:0+4,:], array[12,:])))

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [139]:
comb_rank["pair"] < comb_rank["two pairs"]

True

In [221]:
def check_straights(cards):
    array = cards.card_array
    rank_sum = np.sum(array, axis=1) #asis 1 - suma po rankach #axis 0 suma po kolorach
    for n in range(-1, 10):
        is_straight = True
        for m in range(0, 5):
            if rank_sum[(n+m)%13] < 1:
                is_straight = False
                break
        if is_straight == True:
            if n != -1:
                if max(np.sum(array[n:n+5,:],axis = 0)) == 5:
                    return "straight flush", n+4
            elif max(np.sum(np.vstack((array[0:0+4,:], array[12,:])), axis=0)) == 5:
                return "straight flush", 3
            else:
                return "straight", n+4
    return None, None

In [223]:
print(check_straights(hero_cards("2h3h4h5hAh")))

('straight flush', 3)


In [224]:
def check_above_straight(cards):
    array = cards.card_array
    rank_sum = np.sum(array, axis=1) #asis 1 - suma po rankach #axis 0 suma po kolorach
    color_sum = np.sum(array, axis=0)
    if max(rank_sum) == 4:
        return "quads", np.where(rank_sum == 4)[0][0]
    elif np.size(np.where(rank_sum >= 3)[0]) >= 1 and np.size(np.where(rank_sum >= 2)[0]) >= 2:
        index_of_highest_trips = max(np.where(rank_sum >= 3)[0])
        rank_sum[index_of_highest_trips] = 0
        return "full house", (index_of_highest_trips, max(np.where(rank_sum >= 2)[0]))
    elif max(color_sum) >= 5:
        color_index = np.where(color_sum >= 5)[0]
        index_of_flush = np.where(array[:,color_index] == 1)[0]
        index_of_flush = np.sort(index_of_flush)[::-1]
        return "flush" , (index_of_flush[0], index_of_flush[1], index_of_flush[2])
    else:
        return None, None
    

In [373]:
print(check_above_straight(hero_cards("QdQh4cAc6dQsAs")))

('full house', (10, 12))


In [226]:
def check_below_straight(cards):
    array = cards.card_array
    rank_sum = np.sum(array, axis=1) #asis 1 - suma po rankach #axis 0 suma po kolorach
    color_sum = np.sum(array, axis=0)
    if max(rank_sum) == 3:
        trips_index = max(np.where(rank_sum == 3)[0])
        high_index = np.where(rank_sum == 1)[0]
        high_index = np.sort(high_index)[::-1]
        return "trips", (trips_index, high_index[0], high_index[1])
    elif np.size(np.where(rank_sum == 2)[0]) >= 2:
        pair_index = np.where(rank_sum == 2)[0]
        pair_index = np.sort(pair_index)[::-1]
        out1 = pair_index[0]
        out2 = pair_index[1]
        rank_sum[out1] = 0
        rank_sum[out2] = 0
        high_index = np.where(rank_sum >= 1)[0]
        return "two pair", (out1, out2, max(high_index))
    elif np.size(np.where(rank_sum == 2)[0]) == 1:
        pair_index = np.where(rank_sum == 2)[0][0]
        high_index = np.where(rank_sum == 1)[0]
        high_index = np.sort(high_index)[::-1]
        return "pair", (pair_index, high_index[0], high_index[1], high_index[2])
    else:
        high_index = np.where(rank_sum == 1)[0]
        high_index = np.sort(high_index)[::-1]
        return "high card", (high_index[0], high_index[1], high_index[2], high_index[3], high_index[4])



In [227]:
comb_rank = {"high card" : 0, "pair" : 1, "two pairs" : 2, "trips" : 3, "straight" : 4, "flush" : 5, "full house" : 6, "quads" : 7, "straight flush" : 8}

In [376]:
def check_combo(cards):
    comb, helpers = check_straights(cards)
    if comb == "straight flush":
        return comb, helpers
    if comb == "straight":
        comb2, helpers2 = check_above_straight(cards)
        if comb2 != None:
            return comb2, helpers2
        else:
            return comb, helpers
    if comb == None:
        comb2, helpers2 = check_above_straight(cards)
        if comb2 != None:
            return comb2, helpers2
        comb, helpers = check_below_straight(cards)
        return comb, helpers

In [230]:
print(check_below_straight(hero_cards("QcThAdAc2c8d7h")))

('pair', (12, 10, 8, 6))


In [377]:
print(check_combo(hero_cards("2d3h4c5c6dQsAs")))
print(check_combo(hero_cards("QdQh4cAc6dQsAs")))

('straight', 3)
('full house', (10, 12))


In [409]:
cards = hero_cards()
for n in range(7):
    cards.deal_random()
while check_combo(cards)[0] != "flush":
    for n in range(7):
        cards.deal_random()
print(check_combo(cards))
print(cards.card_array)

('flush', (12, 11, 10))
[[1. 0. 0. 1.]
 [1. 0. 1. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [408]:
cards = hero_cards()
for n in range(7):
    cards.deal_random()

print(check_combo(cards))
print(cards.card_array)

('pair', (10, 7, 5, 4))
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [277]:
print(cards.card_array)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


In [104]:
cards = hero_cards("AhTd","8c9c")
print(cards.card_array)
cards.deal_random()
cards.deal_random()
print(cards.card_array)


Ah
Td
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0.  0.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0.  0.]]


In [75]:
hand_array = np.zeros((13, 4))

print(hand_array)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [76]:
hand_array[card_map['Td'].rank,card_map['Td'].color] += 1
print(hand_array)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [77]:
x = card_map['Td'].index()
hand_array[x] += 1
print(hand_array)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [71]:
# def make_map():
#     columns = ['h', 's', 'd', 'c']
#     rows = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
#     card_map = {}
#     for index_row in range(len(rows)):
#         for index_col in range(len(columns)):
#             key = f"{rows[index_row]}{columns[index_col]}"
#             value = card(12-index_row, index_col, f"{rows[index_row]}{columns[index_col]}")
#             card_map[key] = value
#     return card_map

In [ ]:
# columns = ['h', 's', 'd', 'c']
# rows = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']

# card_map = {}

# for index_row in range(len(rows)):
#     for index_col in range(len(columns)):
#         key = f"{rows[index_row]}{columns[index_col]}"
#         value = card(12-index_row, index_col, f"{rows[index_row]}{columns[index_col]}")
#         card_map[key] = value

# for key, value in card_map.items(): 
#     print(f"{key}: {value.name}")

# value_at_Ah = card_map['Ah']
# print(f"The value at A and h is: {value_at_Ah}")